In [18]:
import numpy as np

In [1]:
!pip install yfinance

Processing c:\users\asus\appdata\local\pip\cache\wheels\57\6d\a3\a39b839cc75274d2acfb1c58bfead2f726c6577fe8c4723f13\multitasking-0.0.9-py3-none-any.whl
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23914 sha256=917614e7a457a695d786c30df21e40fe6f654fe471eeaf86d057f7306c04efab
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\ec\cc\c1\32da8ee853d742d5d7cbd11ee04421222eb354672020b57297
Successfully built yfinance


In [2]:
import yfinance as yf
msft = yf.Ticker("MSFT")

In [4]:
msft.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 181000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. Its Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and managing Microsoft server and desktop solutions; and training and certifica

In [5]:
hist = msft.history(period="5d")

In [6]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-08-19,288.690002,297.470001,288.640015,296.769989,29850500,0,0
2021-08-20,299.720001,305.839996,298.059998,304.359985,40796100,0,0
2021-08-23,303.250000,305.399994,301.850006,304.649994,22830200,0,0
2021-08-24,305.019989,305.649994,302.000000,302.619995,18175800,0,0
2021-08-25,304.299988,304.589996,300.420013,302.010010,19999400,0,0


In [13]:
data_df = yf.download(["AAPL","IBM","GOOG","BP","XOM","COST","GS"],start="2016-01-01", end="2016-12-31")

[*********************100%***********************]  7 of 7 completed


In [115]:
data_df.to_csv('raw_data.csv')

In [35]:
data_df['Adj Close'].pct_change().dropna()

,AAPL,BP,COST,GOOG,GS,IBM,XOM
Date,,,,,,,
2016-01-04,0.000855,-0.006398,-0.012136,-0.022454,-0.017145,-0.012135,-0.006286
2016-01-05,-0.025059,-0.004185,0.002444,0.000998,-0.017219,-0.000735,0.008521
2016-01-06,-0.019570,-0.020369,-0.009254,0.001400,-0.024412,-0.005006,-0.008321
2016-01-07,-0.042205,-0.028713,-0.022909,-0.023170,-0.030735,-0.017090,-0.016006
2016-01-08,0.005288,-0.017669,-0.017504,-0.016410,-0.004131,-0.009258,-0.020202
...,...,...,...,...,...,...,...
2016-12-23,0.001978,0.002981,0.000062,-0.001706,0.003540,-0.002095,-0.001761
2016-12-27,0.006351,0.002431,-0.000185,0.002076,0.002448,0.002579,0.000441
2016-12-28,-0.004264,0.003772,-0.006418,-0.008212,-0.003767,-0.005684,-0.004959


In [62]:
weights=[0.15,0.2,0.2,0.15,0.1,0.15,0.05]

In [63]:
port_daily_return=np.dot(weights,data_df['Adj Close'].pct_change().dropna().T)

In [64]:
port_daily_return.sort()

In [65]:
np.quantile(port_daily_return,q=0.05)

-0.014663853181955372

In [66]:
port_daily_return[port_daily_return<=np.quantile(port_daily_return,q=0.05)]

array([-0.03815698, -0.03013781, -0.02941983, -0.02656794, -0.02118777,
       -0.02036254, -0.01904709, -0.01867242, -0.017415  , -0.01726935,
       -0.01698118, -0.01652026, -0.01544585])

In [67]:
port_daily_return[port_daily_return<=np.quantile(port_daily_return,q=0.05)].mean()

-0.022091078471798555

In [68]:
covariance_matrix=data_df['Adj Close'].pct_change().dropna().cov()

In [74]:
expected_mean=port_daily_return.mean()

In [75]:
expected_mean

0.0006768970446547169

In [79]:
expected_variance=np.dot(np.dot(weights,covariance_matrix),weights)

In [80]:
expected_variance

8.81349130912686e-05

In [107]:
var_95=-1.6449*np.sqrt(expected_variance)+expected_mean

In [108]:
var_95

-0.01476545649011265

In [112]:
from scipy import integrate
def f(x, miu, sigma):
    return x*(1/(np.sqrt(2*np.pi)*sigma))*np.exp(-0.5*((x-miu)**2/sigma**2))
v, err = integrate.quad(f, -np.inf,var_95, args = (expected_mean, np.sqrt(expected_variance)))
print(v/0.05)

-0.01868647903574482
